# IMPORT REQUIRED LIBRARIES

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from imageio import imread

import warnings
warnings.filterwarnings("ignore")

C:\Users\misak\AppData\Local\Temp\ipykernel_24400\2458891310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# IMPORT DATASET

In [2]:
xdf = pd.read_csv("games.csv")

In [3]:
xdf

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1507,Back to the Future: The Game,"Dec 22, 2010",['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...,763,5,223,67
1508,1508,Team Sonic Racing,"May 21, 2019","['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C...",1.5K,49,413,107
1509,1509,Dragon's Dogma,"May 22, 2012",['Capcom'],3.7,210,210,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ...",1.1K,45,487,206
1510,1510,Baldur's Gate 3,"Oct 06, 2020",['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...,269,79,388,602


In [4]:
xdf.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [5]:
xdf.shape

(1512, 14)

1512 rows of data and 14 variables

In [6]:
xdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


# Removing unnecessary columns/variables

In [7]:
dataG = xdf[[
      'Title',
      'Release Date',
      'Number of Reviews',
      'Genres',
      'Summary',
      'Reviews',
      'Rating',
      'Playing',
      'Backlogs',
      'Wishlist',
      'Plays']]
dataG.head()

,Title,Release Date,Number of Reviews,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays
0,Elden Ring,"Feb 25, 2022",3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,3.8K,4.6K,4.8K,17K
1,Hades,"Dec 10, 2019",2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,3.2K,6.3K,3.6K,21K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017",4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2.5K,5K,2.6K,30K
3,Undertale,"Sep 15, 2015",3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,679,4.9K,1.8K,28K
4,Hollow Knight,"Feb 24, 2017",3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2.4K,8.3K,2.3K,21K


DATA CLEANING (Handling of NULL rows)

In [8]:
dataG.isnull().sum()

Title                 0
Release Date          0
Number of Reviews     0
Genres                0
Summary               1
Reviews               0
Rating               13
Playing               0
Backlogs              0
Wishlist              0
Plays                 0
dtype: int64

In [9]:
dataG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1512 non-null   object 
 1   Release Date       1512 non-null   object 
 2   Number of Reviews  1512 non-null   object 
 3   Genres             1512 non-null   object 
 4   Summary            1511 non-null   object 
 5   Reviews            1512 non-null   object 
 6   Rating             1499 non-null   float64
 7   Playing            1512 non-null   object 
 8   Backlogs           1512 non-null   object 
 9   Wishlist           1512 non-null   object 
 10  Plays              1512 non-null   object 
dtypes: float64(1), object(10)
memory usage: 130.1+ KB


In [10]:
#removing rows with NULL value
#for example, games with no reviews

In [11]:
#drop all NULL values
dataG=dataG.dropna()

#reset index rows of dataframe to avoid non-consecutive index values
dataG=dataG.reset_index(drop=True)

print(f"Shape of new dataset is: {dataG.shape}")

Shape of new dataset is: (1499, 11)


In [12]:
#there were 13 rows wil null values
#Double checking if there are any null values left in the dataset
dataG.isnull().values.any()


False

# DATA CLEANING

# Changing of String release of Date to integer datatype with year only

In [13]:
#Extract year from release data and drop Release date column
import datetime

def extract_year(date_str):
    try:
        date_object = datetime.datetime.strptime(date_str, "%b %d, %Y")
        return date_object.year
    except ValueError:
        return pd.NA  # Handle potential format errors (optional)

dataG['Year'] = dataG['Release Date'].apply(extract_year)
#dataG.drop('Release Date',axis=1) 
data=dataG.drop('Release Date',axis=1) 
data.head()

,Title,Number of Reviews,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays,Year
0,Elden Ring,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,3.8K,4.6K,4.8K,17K,2022
1,Hades,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,3.2K,6.3K,3.6K,21K,2019
2,The Legend of Zelda: Breath of the Wild,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2.5K,5K,2.6K,30K,2017
3,Undertale,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,679,4.9K,1.8K,28K,2015
4,Hollow Knight,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2.4K,8.3K,2.3K,21K,2017


In [14]:
data.isnull().values.any()

True

In [15]:
#Release date column contain strings with "releases on TBD" which means the game has not been released yet
#extracting the year from this column cause this string value to be null and now we also have to remove the rows containing it
#drop all NULL values
data=data.dropna()

#reset index rows of datframe to avoid non-consecutive index values
data=data.reset_index(drop=True)

#convert date column from string to int
data['Year']=data['Year'].astype(int)
#df['points'] = df['points'].astype(str).astype(int)

print(f"Shape of new dataset is: {data.shape}")


Shape of new dataset is: (1497, 11)


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1497 non-null   object 
 1   Number of Reviews  1497 non-null   object 
 2   Genres             1497 non-null   object 
 3   Summary            1497 non-null   object 
 4   Reviews            1497 non-null   object 
 5   Rating             1497 non-null   float64
 6   Playing            1497 non-null   object 
 7   Backlogs           1497 non-null   object 
 8   Wishlist           1497 non-null   object 
 9   Plays              1497 non-null   object 
 10  Year               1497 non-null   int32  
dtypes: float64(1), int32(1), object(9)
memory usage: 122.9+ KB


# Changing of string values to integer value for numerical variables

In [17]:
# Converts the numbers with K to multiple by 1000 and convert into whole integers since K = 1000. Eg. 1K = 1 * 1000
def extract_value(value_str):
    if value_str.upper().endswith('K'):
        value_str = value_str[:-1]
        value_num = float(value_str) * 1000
    else:
        value_num = float(value_str)
    return value_num


# Combining Backlogs and Wishlish to make a new column "Interested" 

In [18]:
games_clean = data.copy()

#replaces K for 1000
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace("K","000")
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace(".","")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace("K","000")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace(".","")

games_clean[['Backlogs', 'Wishlist']] = games_clean[['Backlogs', 'Wishlist']].apply(pd.to_numeric)

#combines column backlogs and wishlist
#for people who are interested
games_clean['Interested'] = games_clean["Backlogs"] + games_clean["Wishlist"]
games_clean['Playing']=games_clean['Playing'].apply(extract_value)
games_clean['Total Plays']=games_clean['Plays'].apply(extract_value)
games_clean['Review_count']=games_clean['Number of Reviews'].apply(extract_value)

#convert to float (except year) for consistency
games_clean['Playing']=games_clean['Playing'].astype(float)
games_clean['Interested']=games_clean['Interested'].astype(float)
games_clean['Total Plays']=games_clean['Total Plays'].astype(float)
games_clean['Review_count']=games_clean['Review_count'].astype(float)

#removes Playing, Backlogs, and Wishlist from df
games_clean.pop("Backlogs")
games_clean.pop("Wishlist")
#games_clean.pop("Playing")
games_clean.pop("Plays")
games_clean.pop("Number of Reviews")


games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Playing,Year,Interested,Total Plays,Review_count
0,Elden Ring,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,3800.0,2022,94000.0,17000.0,3900.0
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,3200.0,2019,99000.0,21000.0,2900.0
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2500.0,2017,31000.0,30000.0,4300.0
3,Undertale,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,679.0,2015,67000.0,28000.0,3500.0
4,Hollow Knight,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2400.0,2017,106000.0,21000.0,3000.0


In [19]:
# Check if there are any other NULL values
games_clean.info()
games_clean.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1497 non-null   object 
 1   Genres        1497 non-null   object 
 2   Summary       1497 non-null   object 
 3   Reviews       1497 non-null   object 
 4   Rating        1497 non-null   float64
 5   Playing       1497 non-null   float64
 6   Year          1497 non-null   int32  
 7   Interested    1497 non-null   float64
 8   Total Plays   1497 non-null   float64
 9   Review_count  1497 non-null   float64
dtypes: float64(5), int32(1), object(4)
memory usage: 111.2+ KB


False

# Adding columns from another column/variable for easier usage of data

In [20]:
split_Genres = games_clean.copy()

unique_genres = set()
for genres_str in split_Genres['Genres']:
    genres_list = eval(genres_str)  # Convert string representation of list to actual list
    unique_genres.update(genres_list)

# Create new columns for each unique genre
for genre in unique_genres:
    split_Genres[genre] = 0

# Iterate over each row to populate the new genre columns
for index, row in games_clean.iterrows():
    genres_list = eval(row['Genres'])  # Convert string representation of list to actual list
    for genre in genres_list:
        split_Genres.at[index, genre] = 1
        
#drop genre column
#split_Genres.pop("Genres")

In [21]:
#Check if there are any empty info in the data.
games_clean.isna().sum()

Title           0
Genres          0
Summary         0
Reviews         0
Rating          0
Playing         0
Year            0
Interested      0
Total Plays     0
Review_count    0
dtype: int64

# Cleaning text from random unicode characters (In this case is the []), and keeping only letters, numbers, spaces. Applies to summary and reviews

In [22]:
def decode(column):
    column = column.str.decode("unicode_escape")\
    .str.replace(r'[^a-zA-Z1-9\ ]', '', regex=True).str.strip()
    return column

In [23]:
games_clean.loc[:,"Summary"] = decode(games_clean["Summary"])
games_clean.loc[:,"Reviews"] = decode(games_clean["Reviews"])

# This part of the code removes ONLY completely non-english reviews, basically games with reviews that does not contain english. Games with reviews with english reviews will still stay.

# It detects which reviews are in English. Puts the english reviews in a new dataframe called english_reviews

In [24]:
# Function to detect the language of a review using langid with confidence threshold
def detect_language_with_langdetect(line): 
    from langdetect import detect_langs
    try: 
        langs = detect_langs(line) 
        for item in langs: 
            # The first one returned is usually the one that has the highest probability
            return item.lang, item.prob 
    except: return "err", 0.0 

# Apply language detection to each review and add a 'IsEnglish' column to the DataFrame
games_clean['LanguageDetection'] = games_clean['Reviews'].apply(detect_language_with_langdetect)

#Filter out non-English reviews based on the threshold
english_reviews = games_clean[games_clean['LanguageDetection'].apply(lambda x: x[0] == 'en' and x[1] >= 0.8)]
games_clean = english_reviews

# Output the results with language and confidence percentages
# for idx, (lang, confidence) in enumerate(english_reviews['LanguageDetection']):
#     print(f"Review {idx + 1}: Language={lang}, Confidence={confidence:.2%}")

# print(games_clean)

# Resets the index after cleaning of random unicode characters is done

In [25]:
games_clean.reset_index(inplace=True, drop=True)
games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Playing,Year,Interested,Total Plays,Review_count,LanguageDetection
0,Elden Ring,"['Adventure', 'RPG']",Elden Ring is a fantasy action and open world ...,The first playthrough of elden ring is one of ...,4.5,3800.0,2022,94000.0,17000.0,3900.0,"(en, 0.9999968988237165)"
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']",A roguelite hack and slash dungeon crawler in ...,convinced this is a roguelike for people who d...,4.3,3200.0,2019,99000.0,21000.0,2900.0,"(en, 0.9999986706149238)"
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']",The Legend of Zelda Breath of the Wild is the ...,This game is the game that is not CSGO that I ...,4.4,2500.0,2017,31000.0,30000.0,4300.0,"(en, 0.8571399437153016)"
3,Undertale,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",A small child falls into the Underground where...,soundtrack is tied for 1 with nier automata a...,4.2,679.0,2015,67000.0,28000.0,3500.0,"(en, 0.9999976399717436)"
4,Hollow Knight,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,this games worldbuilding is incredible with it...,4.4,2400.0,2017,106000.0,21000.0,3000.0,"(en, 0.9999959468641114)"
